### Importing Libraries

In [1]:
!curl https://download.mosek.com/stable/8.1.0.82/mosektoolslinux64x86.tar.bz2 --output /root/mosektoolslinux64x86.tar.bz2
!cd /root/; tar xvf /root/mosektoolslinux64x86.tar.bz2
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/EE6332/mosek.lic /root/mosek/
!pip install git+https://github.com/convexengineering/gpkit
!pip show gpkit

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.3M  100 40.3M    0     0  52.3M      0 --:--:-- --:--:-- --:--:-- 52.3M
mosek/8/tools/examples/python/parameters.py
mosek/8/toolbox/examples/cqo1.m
mosek/8/toolbox/r2014a/mskqpopt.m
mosek/8/tools/examples/fusion/java/breaksolver.java
mosek/8/toolbox/r2014a/mskcheck.m
mosek/8/tools/examples/fusion/cxx/lo1.cc
mosek/8/tools/examples/data/dgo.f
mosek/8/tools/examples/dotnet/scopt1.cs
mosek/8/tools/examples/dotnet/solvebasis.cs
mosek/8/toolbox/r2014aom/mskerrmsg.m
mosek/8/toolbox/examples/simple.m
mosek/8/tools/examples/java/solutionquality.java
mosek/8/tools/examples/fusion/dotnet/production.cs
mosek/8/tools/examples/fusion/java/Makefile
mosek/8/tools/examples/fusion/python/lo1.py
mosek/8/tools/examples/fusion/java/duality.java
mosek/8/tools/platform/linux64x86/bin/libcilkrts.so.5
mosek/8/toolbox/examples/qo2.m
mosek/8/tools/ex

In [2]:
from pylab import *
from gpkit import VectorVariable, Variable, Model, units

Found no installed solvers, beginning a build.
# Building GPkit version 1.1
# Moving to the directory from which GPkit was imported.

Attempting to find and build solvers:

# Looking for `mosek_cli`
#   (A "success" is if mskexpopt complains that
#    we haven't specified a file for it to open.)
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##
# Looks like `mskexpopt` was not found in the default PATH,
#  so let's try locating that binary ourselves.
#   Adding /root/mosek/8/tools/platform/linux64x86/bin to the PATH
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##

Found mosek_cli in /root/mosek/8/tools/platform/linux64x86/bin

# Looking for `mosek_conif`
#   Trying to import mosek...
# Did not find
# mosek_conif

# Looking for `cvxopt`
#   Trying to import cvxopt...


.


Found cvxopt in the default PYTHONPATH

Found the following solvers: mosek_cli, cvxopt
#     Replacing directory env


.........................E......................................................................................................................................................................................
ERROR: test_vector_sweep (gpkit.tests.t_sub.TestModelSubs.test_vector_sweep)
Test sweep involving VectorVariables
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gpkit/tests/t_sub.py", line 205, in test_vector_sweep
    sol = m.solve(verbosity=0)
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gpkit/constraints/prog_factories.py", line 123, in solvefn
    constants, sweep, linked = parse_subs(self.varkeys, self.substitutions)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gpkit/nomials/substitution.py", line 25, in parse_subs
    append_sub(sub, keys, constants, sweep, l


GPkit is now installed with solver(s) ['mosek_cli', 'cvxopt']
To incorporate new solvers at a later date, run `gpkit.build()`.

If any tests didn't pass, please post the output above
(starting from "Found no installed solvers, beginning a build.")
to gpkit@mit.edu or https://github.com/convexengineering/gpkit/issues/new
so we can prevent others from having these errors.

The same goes for any other bugs you encounter with GPkit:
send 'em our way, along with any interesting models, speculative features,
comments, discussions, or clarifications you feel like sharing.

Finally, we hope you find our documentation (https://gpkit.readthedocs.io/)
and engineering-design models (https://github.com/convexengineering/gplibrary/)
to be useful resources for your own applications.

Enjoy!



### Classes Required

In [3]:
import re
from collections import defaultdict, deque

class Gate:
    def __init__(self, gate_type, name, output, inputs):
        self.name = name
        self._type = gate_type
        self.output = output
        self.inputs = inputs
        self.logical_effort = self.compute_logical_effort()
        self.parasitic_effort = self.compute_parasitic_effort()
        self.input_cap = self.compute_input_capacitance()  # Corrected: now called after setting needed fields
        self.size_options = [i for i in range(1,65)]
        self.current_size = 64  # Initial default size

    def compute_logical_effort(self):
        # Logical effort for different gates based on the number of inputs (fan-in)
        le_dict = {'NAND': {2: 4/3, 3: 5/3, 4: 2},'NOR': {2: 5/3, 3: 7/3, 4: 3},'NOT': {1: 1},}

        # Check the number of inputs for the gate and return the appropriate logical effort
        gate_type = self._type.upper()
        num_inputs = len(self.inputs)

        if gate_type in le_dict and num_inputs in le_dict[gate_type]:
            return le_dict[gate_type][num_inputs]
        return 1  # Default for undefined gate types

    def compute_parasitic_effort(self):
        return len(self.inputs)

    def compute_input_capacitance(self):
        return 3*self.compute_logical_effort()

    def get_primary_input_to_gate_map(self):
        input_to_gates = defaultdict(list)
        primary_inputs = self.find_primary_inputs()
        for inp in primary_inputs:
            if inp in self.reverse_graph:
                for out_node in self.reverse_graph[inp]:
                    gate = self.node_to_gate.get(out_node)
                    if gate:
                        input_to_gates[inp].append(gate)
        return dict(input_to_gates)

    def __repr__(self):
        return (f"Gate(name={self.name}, type={self._type}, output={self.output}, "
                f"inputs={self.inputs}, LE={self.logical_effort}, PE={self.parasitic_effort})")

In [4]:
class VerilogParser:
    def __init__(self, verilog_code):
        self.graph = defaultdict(list)
        self.reverse_graph = defaultdict(list)
        self.nodes = set()
        self.node_to_gate = {}
        self.gate_type = {}
        self.gates = {}  # New: name -> Gate instance
        self.parse_verilog(verilog_code)

    def parse_verilog(self, verilog_code):
        lines = [line.strip() for line in verilog_code.strip().splitlines() if line.strip()]
        gate_pattern = re.compile(r'(nand|nor|not)\s+(\w+)\s*\(([^)]+)\);')

        for line in lines:
            match = gate_pattern.match(line)
            if match:
                gate_type, gate_name, pin_list = match.groups()
                pins = [pin.strip() for pin in pin_list.split(',')]
                output = pins[0]
                inputs = pins[1:]

                self.node_to_gate[output] = gate_name
                self.gate_type[gate_name] = gate_type
                self.gates[gate_name] = Gate(gate_type, gate_name, output, inputs)

                self.nodes.add(output)
                self.graph[output].extend(inputs)
                for inp in inputs:
                    self.reverse_graph[inp].append(output)
                    self.nodes.add(inp)

    def find_primary_outputs(self):
        return [node for node in self.nodes if node not in self.reverse_graph]

    def find_primary_inputs(self):
        return [node for node in self.nodes if node not in self.graph]

    def print_fanout_list(self):
        fanout_list = defaultdict(list)
        for output_node, gate_name in self.node_to_gate.items():
            if output_node in self.reverse_graph:
                for fanout_node in self.reverse_graph[output_node]:
                    if fanout_node in self.node_to_gate:
                        fanout_list[gate_name].append(self.node_to_gate[fanout_node])
        return {gate: ', '.join(fanouts) for gate, fanouts in fanout_list.items() if fanouts}

    def get_all_gates(self):
        return list(self.gates.keys())

    def get_primary_inputs(self):
        return [n for n in self.graph.nodes if not self.graph.nodes[n]['inputs']]  # Buggy

    def get_fanout_dict(self):
        fanout_dict = {gate: [] for gate in self.gates}
        for gate, gate_obj in self.gates.items():
            for inp in gate_obj.inputs:
                if inp in self.gates:
                    fanout_dict[inp].append(gate)
        return fanout_dict

### Constraint Former

In [5]:
from gpkit import Variable, VectorVariable
import numpy as np

def constraints_former(mode, parser, T_spec=None, a=1):
    gate_nodes = parser.get_all_gates()
    fanout_dict_raw = parser.print_fanout_list()
    fanout_dict = {
        gate: [fout for fout in fanouts.split(', ') if fout]
        for gate, fanouts in fanout_dict_raw.items()
    }

    primary_inputs = parser.find_primary_inputs()
    pog = []

    gate_index = {gate: idx for idx, gate in enumerate(gate_nodes)}
    N = len(gate_nodes)
    Gsize = VectorVariable(N, 'x')
    T = VectorVariable(N, 'T')

    gate_caps = {gate: parser.gates[gate].input_cap for gate in gate_nodes}
    g_vals = np.array([parser.gates[gate].logical_effort for gate in gate_nodes])
    p_vals = np.array([parser.gates[gate].parasitic_effort for gate in gate_nodes])
    input_caps = np.array([gate_caps[gate] for gate in gate_nodes])

    C_load_size = 1000
    C_load = 3 * C_load_size
    tau = 5
    max_gate_size = 64

    constraints = []
    fanin_indices = {gate: [] for gate in gate_nodes}

    # Initialize fanout_caps as a numpy array of zeros with size N
    fanout_caps = np.zeros(N, dtype=object)

    for i, gate in enumerate(gate_nodes):
        fanouts = fanout_dict.get(gate, [])
        if fanouts:
            fanout_caps[i] = sum(
                gate_caps[fout] * Gsize[gate_index[fout]]
                for fout in fanouts if fout in gate_caps
            )
        else:
            fanout_caps[i] = C_load
            pog.append(gate)

    for gate, fanouts in fanout_dict.items():
        for fout in fanouts:
            if fout in fanin_indices:
                fanin_indices[fout].append(gate)

    delays_expr = (g_vals * fanout_caps) / (input_caps * Gsize) + p_vals
    delays = list(delays_expr)

    for i, gate in enumerate(gate_nodes):
        if fanin_indices[gate]:
            for fanin in fanin_indices[gate]:
                constraints.append(T[i] >= T[gate_index[fanin]] + delays[i])
        else:
            constraints.append(T[i] >= delays[i])

    max_pi_load = 150
    graph = {gate: {'inputs': parser.gates[gate].inputs} for gate in gate_nodes}
    pi_to_gates = {pi: [] for pi in primary_inputs}
    for gate in gate_nodes:
        for inp in graph[gate]['inputs']:
            if inp in pi_to_gates:
                pi_to_gates[inp].append(gate)

    for pi, gates in pi_to_gates.items():
        if gates:
            total_load = np.sum(gate_caps[g] * Gsize[gate_index[g]] for g in gates)
            constraints.append(total_load <= max_pi_load)

    constraints += [Gsize >= 1, Gsize <= max_gate_size]

    final_outputs = [i for i, gate in enumerate(gate_nodes) if not fanout_dict.get(gate)]
    if mode == 0:
        Tmax = Variable("Tmax")
        for i in final_outputs:
            constraints.append(Tmax >= T[i])
        return Tmax, constraints, gate_nodes, pog
    elif mode == 1:
        for i in final_outputs:
            constraints.append(T[i] <= a*T_spec)
        return Gsize, constraints, gate_nodes, pog



### Solver for Twall

In [6]:
def optimize_critical_path_delay(Varmin, constraints, gate_nodes,pog):
    # Solve the model
    N = len(gate_nodes)
    m = Model(Varmin, constraints)
    sol = m.solve(verbosity=1)
    tau = 5

    Tmax_val = sol["variables"]["Tmax"]
    Tmax_ps = Tmax_val * tau
    sizes = {gate_nodes[i]: float(sol["variables"]["x"][i]) for i in range(N)}
    arrival_times = {gate_nodes[i]: float(sol["variables"]["T"][i]) for i in range(N)}

    return Tmax_val, Tmax_ps, sizes, arrival_times, pog

### Solver for Area

In [7]:
def optimize_area(Varmin, constraints, gate_nodes, pog):
    N = len(gate_nodes)
    model = Model(Varmin.sum(), constraints)
    try:
        sol = model.solve(verbosity=1)
    except Exception as e:
        #print("[optimize_area] Solver failed due to infeasibility. Here are the constraints:")
        #for constr in model.flat():
          #  print(constr)
        return None, None

    sizes = {gate_nodes[i]: float(sol["variables"]["x"][i]) for i in range(N)}
    return sizes, sum(list(sizes.values()))


### Continuous Time First.

In [9]:
filename = "c432.txt"
with open(filename, 'r') as file:
    verilog_code = file.read()

# Process each file

print(f"Processing: {filename}")
parser = VerilogParser(verilog_code)
Varmin, constraints, gate_nodes, pog = constraints_former(0,parser)

Processing: c432.txt


<ipython-input-5-3c3e5f8ce547>:39: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  fanout_caps[i] = sum(
<ipython-input-5-3c3e5f8ce547>:72: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  total_load = np.sum(gate_caps[g] * Gsize[gate_index[g]] for g in gates)


In [10]:
Twall,_,_,_,_ = optimize_critical_path_delay(Varmin, constraints, gate_nodes,pog)
print(float(Twall))

Using solver 'mosek_cli'
 for 459 free variables
  in 880 posynomial inequalities.
Solving took 0.263 seconds.
153.471518104049


In [11]:
a = 1.25

In [12]:
VarA, constraints_area, gate_nodes, pog = constraints_former(1,parser,Twall,a)

<ipython-input-5-3c3e5f8ce547>:39: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  fanout_caps[i] = sum(
<ipython-input-5-3c3e5f8ce547>:72: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  total_load = np.sum(gate_caps[g] * Gsize[gate_index[g]] for g in gates)


In [13]:
size_dict, _ = optimize_area(VarA, constraints_area, gate_nodes, pog)

Using solver 'mosek_cli'
 for 458 free variables
  in 880 posynomial inequalities.
Solving took 0.163 seconds.


### Multimove and Heuristic Fused

In [16]:
from gpkit import Variable, VectorVariable, Model

import math

def check_post_rounding_timing(parser, sizes_ceil, Tmax_val, tau=5.0):
    gate_nodes = parser.get_all_gates()
    fanout_dict_raw = parser.print_fanout_list()
    fanout_dict = {
        gate: [fout for fout in fanouts.split(', ') if fout]
        for gate, fanouts in fanout_dict_raw.items()
    }
    gate_nodes = parser.get_all_gates()
    fanout_dict_raw = parser.print_fanout_list()
    fanout_dict = {
        gate: [fout for fout in fanouts.split(', ') if fout]
        for gate, fanouts in fanout_dict_raw.items()
    }
    ge = {gname: parser.gates[gname].compute_logical_effort() for gname in gate_nodes}
    p = {gname: parser.gates[gname].compute_parasitic_effort() for gname in gate_nodes}
    C_in = {gname: parser.gates[gname].input_cap for gname in gate_nodes}
    primary_inputs = parser.find_primary_inputs()

    # Fanin map
    fanin_map = {g: [] for g in gate_nodes}
    for gate, fanouts in fanout_dict.items():
        for fout in fanouts:
            fanin_map[fout].append(gate)

    # Arrival time and delay computation
    arrival_times = {}
    delays = {}
    C_load_size = 1000
    C_load = 3 * C_load_size
    Tmax_ps = Tmax_val * tau
    Tmax_allowed = 1.3 * Tmax_ps

    # Topological order (assumes acyclic)
    visited = set()
    topo_order = []

    def dfs(g):
        if g in visited:
            return
        visited.add(g)
        for fout in fanout_dict.get(g, []):
            dfs(fout)
        topo_order.append(g)

    for g in gate_nodes:
        dfs(g)

    topo_order.reverse()

    for gate in topo_order:
        # Fanout cap
        fanouts = fanout_dict.get(gate, [])
        if fanouts:
            Cfan = sum(C_in[f] * sizes_ceil[f] for f in fanouts)
        else:
            Cfan = C_load

        #print(type(ge),type(C_in),type(sizes_ceil),type(p))
        delay = (ge[gate] * Cfan) / (C_in[gate] * sizes_ceil[gate]) + p[gate]
        delays[gate] = delay

        if fanin_map[gate]:
            arrival_times[gate] = max(arrival_times[fanin] + delays[gate] for fanin in fanin_map[gate])
        else:
            arrival_times[gate] = delays[gate]

    # Critical path delay (post rounding)
    max_gate = max(arrival_times, key=arrival_times.get)
    critical_path_delay = arrival_times[max_gate]

    # Trace back critical path
    critical_path = []
    g_now = max_gate
    while True:
        critical_path.append(g_now)
        fanins = fanin_map[g_now]
        if not fanins:
            break
        g_now = max(fanins, key=lambda x: arrival_times[x])
    critical_path.reverse()

    # Violations
    timing_violations = {
        gate: atime for gate, atime in arrival_times.items() if atime > Tmax_allowed/5
    }

    size_violations = {
        gate: size for gate, size in sizes_ceil.items() if size > 64
    }

    # Primary input shared load
    pi_to_gates = {pi: [] for pi in primary_inputs}
    for gate in gate_nodes:
        for inp in parser.gates[gate].inputs:
            if inp in primary_inputs:
                pi_to_gates[inp].append(gate)

    pi_violations = {}
    for pi, gates in pi_to_gates.items():
        load = sum(C_in[g] * sizes_ceil[g] for g in gates)
        if load > 150:
            pi_violations[pi] = load

    # Final check: ensure consistency
    #assert critical_path_delay >= Tmax_ps, (
     #   f"Critical path delay {critical_path_delay:.2f} ps is less than Twall {Tmax_ps:.2f} ps"
    #)

    return {
        "sizes_ceil": sizes_ceil,
        "arrival_times": arrival_times,
        "delays": delays,
        "timing_violations": timing_violations,
        "size_violations": size_violations,
        "pi_violations": pi_violations,
        "Tmax_ps": Tmax_ps,
        "Tmax_allowed": Tmax_allowed,
        "critical_path": critical_path,
        "critical_path_delay": critical_path_delay
    }


In [19]:
class Network:
    def __init__(self, gates, parser, pog, Twall):
        self.gates = gates
        self.parser = parser

    def estimate_load(self, gate):
        fanouts = self.parser.reverse_graph.get(gate.output, [])
        return sum(
            self.parser.gates[self.parser.node_to_gate[f]].current_size *
            self.parser.gates[self.parser.node_to_gate[f]].input_cap
            for f in fanouts if f in self.parser.node_to_gate
        ) or 3000

    def compute_delays(self):
        delays = {}
        for gate_name, gate in self.gates.items():
            load = self.estimate_load(gate)
            delay = (gate.logical_effort * load / (gate.current_size * gate.input_cap)) + gate.parasitic_effort
            delays[gate_name] = delay
        return delays

    def compute_arrival_times(self, delays):
        arrival = {}

        def dfs(gate_name):
            if gate_name in arrival:
                return arrival[gate_name]
            gate = self.gates[gate_name]
            max_pred_time = 0
            for inp in gate.inputs:
                if inp in self.parser.node_to_gate:
                    pred = self.parser.node_to_gate[inp]
                    pred_time = dfs(pred)
                    max_pred_time = max(max_pred_time, pred_time)
            arrival[gate_name] = delays[gate_name] + max_pred_time
            return arrival[gate_name]

        for gate in self.gates:
            dfs(gate)
        return arrival

    def compute_required_arrival_times(self, delay_constraint, pog, Twall):
        delays = self.compute_delays()
        rat = {}

        # Initialize RATs at primary outputs
        for po in pog:
            if po in self.parser.node_to_gate:
                gate_name = self.parser.node_to_gate[po]
                rat[gate_name] = 1.3 * Twall

        def dfs_rats(gate_name):
            if gate_name in rat:
                return rat[gate_name]
            gate = self.gates[gate_name]
            fanouts = self.parser.reverse_graph.get(gate.output, [])
            min_rat = 1.3 * Twall
            for fo in fanouts:
                if fo in self.parser.node_to_gate:
                    fo_gate = self.parser.node_to_gate[fo]
                    fo_rat = dfs_rats(fo_gate)
                    min_rat = min(min_rat, fo_rat - delays[fo_gate])
            rat[gate_name] = min_rat
            return rat[gate_name]

        for gate_name in self.gates:
            dfs_rats(gate_name)

        return rat

    def compute_slacks(self, arrival_times, rat_times):
        return {gate: rat_times[gate] - arrival_times[gate] for gate in self.gates}

    def extract_subnetwork(self, gate_name, depth=1):
        visited, frontier = set(), {gate_name}
        for _ in range(depth):
            next_frontier = set()
            for g in frontier:
                gate = self.gates[g]
                for inp in gate.inputs:
                    if inp in self.parser.node_to_gate:
                        next_frontier.add(self.parser.node_to_gate[inp])
                for fout in self.parser.reverse_graph.get(gate.output, []):
                    if fout in self.parser.node_to_gate:
                        next_frontier.add(self.parser.node_to_gate[fout])
            visited.update(frontier)
            frontier = next_frontier - visited
        return visited | frontier

    def evaluate_local_cost(self, subnetwork, rat_times):
        delays = self.compute_delays()
        arrival = self.compute_arrival_times(delays)
        slacks = self.compute_slacks(arrival, rat_times)
        S = min(slacks.values())

        if S < 0:
            return (float('inf'), float('inf'))

        area = sum(self.gates[g].current_size for g in subnetwork)
        return (-S, area)

    def get_perturbed_nodes(self, moved_gate_names):
        perturbed = set()
        for name in moved_gate_names:
            perturbed |= self.extract_subnetwork(name, depth=1)
        return perturbed

    def go_to_local_min(self, rat_times):
        import time
        self.rat_times = rat_times
        update = set(self.gates.keys())
        converged = False
        iteration = 0
        start_time = time.time()
        while not converged and iteration < 16 and time.time()-start_time<1800:
            moves = {}
            for gate_name in update:
                gate = self.gates[gate_name]
                original_size = int(gate.current_size)
                subnetwork = self.extract_subnetwork(gate_name, depth=1)

                best_move = original_size
                best_cost = self.evaluate_local_cost(subnetwork, rat_times)

                for size in sorted(gate.size_options):
                    size = int(size)
                    if size == original_size:
                        continue

                    gate.current_size = size
                    cost = self.evaluate_local_cost(subnetwork, rat_times)
                    gate.current_size = original_size

                    if cost is not None and cost < best_cost:
                        best_cost = cost
                        best_move = size

                if best_move != original_size:
                    moves[gate_name] = best_move

            if not moves:
                converged = True
            else:
                for gname, size in moves.items():
                    self.gates[gname].current_size = size
                update = self.get_perturbed_nodes(moves.keys())
                iteration += 1
            def sizes_ceiler(sizes,pi_to_gates):
                import math
                sizes_ceil = {}
                for g,s in sizes.items():
                    #print(pi_to_gates)
                    if g not in pog:
                        sizes_ceil[g] = math.floor(s)
                    else:
                        sizes_ceil[g] = math.ceil(s)
                return sizes_ceil
            if not converged and time.time()-start_time>1800:
                import random
                import math
                import copy
                sizes_ceil = {}
                pi_to_gates = {pi: [] for pi in parser.find_primary_inputs()}
                for g in update:
                    if [g] not in pi_to_gates.values():
                        sizes_ceil[g] = math.ceil(parser.gates[g].current_size)
                    else:
                        sizes_ceil[g] = math.floor(parser.gates[g].current_size)
                max_T = 1.3 * Twall
                lambda_penalty = 1000
                initial_temperature = 100.0
                cooling_rate = 0.99
                min_temperature = 1e-3
                max_iterations = 1000

                num_gates = len(gate_nodes)
                initial_solution = sizes_ceil

                def cost_function(parser, sizes_ceil, Twall,gate_nodes):
                    results = check_post_rounding_timing(parser, sizes_ceil, Twall)
                    delay = results["critical_path_delay"]
                    if delay < Twall:
                        penalty = (Twall - delay)**2
                    elif delay > max_T:
                        penalty = (delay - max_T)**2
                    else:
                        penalty = 0
                    area = sum(list(sizes_ceil.values()))
                    return area + lambda_penalty * penalty, delay, area

                def perturb(sizes_ceil):
                    new_solution = sizes_ceil.copy()
                    move_list = []
                    gate_idx = random.choice(gate_nodes)
                    perturb_options = [-2,-1,0]
                    delta = random.choice(perturb_options)

                    old_size = new_solution[gate_idx]
                    new_size = old_size + delta

                    # Clamp new size to legal range
                    if 1<=new_size<=64:
                        new_solution[gate_idx] = new_size
                        move_list.append((gate_idx, delta))  # record the move
                    else:
                        new_solution[gate_idx] = old_size
                        perturb(sizes_ceil)
                    return new_solution, move_list


                def simulated_annealing(parser,initial_solution,Twall,gate_nodes,a):
                    current_solution = initial_solution.copy()
                    best_solution = current_solution.copy()

                    current_cost, current_delay, current_area = cost_function(parser,current_solution,Twall,gate_nodes)
                    best_cost = current_cost

                    temperature = initial_temperature
                    iteration = 0

                    while temperature > min_temperature and iteration < max_iterations:
                        new_solution, move = perturb(current_solution)
                        new_cost, new_delay, new_area = cost_function(parser,new_solution,Twall,gate_nodes)

                        cost_diff = new_cost - current_cost

                        # Accept if better or with probability exp(-delta/T)
                        if cost_diff < 0 or random.random() < math.exp(-cost_diff / temperature):
                            current_solution = new_solution
                            current_cost = new_cost
                            current_delay = new_delay
                            current_area = new_area

                            if new_cost < best_cost:
                                best_solution = new_solution
                                best_cost = new_cost

                        # Cool down
                        temperature *= cooling_rate
                        iteration += 1

                        if iteration % 100 == 0 or temperature <= min_temperature:
                            print(f"Iter {iteration}: Delay={current_delay:.2f}, Area={current_area}, Cost={current_cost:.2f}")

                    return best_solution, best_cost


                def repeater(parser, sizes_ceil, Twall, gate_nodes, a, max_repeats=11):
                    best_overall_sol, best_overall_cost = simulated_annealing(parser, sizes_ceil, Twall, gate_nodes, a)
                    for repeat in range(max_repeats):
                        VarA, constraints_area, gate_nodes,_, pog = constraints_former(1, parser, Twall, 1.2 +  a)
                        sizes, area = optimize_area(VarA, constraints_area, gate_nodes, pog)
                        sizes_ceil = sizes_ceiler(sizes,pi_to_gates)

                        new_sol, new_cost = simulated_annealing(parser, sizes_ceil, Twall, gate_nodes, a)

                        if new_cost < best_overall_cost:
                            best_overall_cost = new_cost
                            best_overall_sol = new_sol
                        else:
                            print(f"No improvement in iteration {repeat + 1}, stopping.")
                            break  # Stop since there's no improvement
                        a+=0.01
                    return best_overall_sol, best_overall_cost



                best_sol, best_cost = repeater(parser,sizes_ceil,Twall,gate_nodes,0)
                print(sizes_ceil)

        print(f"\nConverged in {iteration} iterations.")

    def greedy_slack_adjustment(self, slacks, rat_times):
        for gate_name, slack in slacks.items():
            gate = self.gates[gate_name]
            original_size = int(gate.current_size)
            possible_sizes = sorted([int(s) for s in gate.size_options])

            if slack > 0.15 * rat_times[gate_name]:
                # Try to downsize if slack is significantly positive
                smaller_sizes = [s for s in possible_sizes if s < original_size]
                for s in reversed(smaller_sizes):
                    gate.current_size = s
                    delays = self.compute_delays()
                    arrival = self.compute_arrival_times(delays)
                    new_slack = self.compute_slacks(arrival, rat_times)[gate_name]
                    if new_slack >= 0:
                        break  # Accept the smaller size
                    gate.current_size = original_size  # Revert if timing violated

            elif slack < 0:
                # Try to upsize to fix timing
                larger_sizes = [s for s in possible_sizes if s > original_size]
                for s in larger_sizes:
                    gate.current_size = s
                    delays = self.compute_delays()
                    arrival = self.compute_arrival_times(delays)
                    new_slack = self.compute_slacks(arrival, rat_times)[gate_name]
                    if new_slack >= 0:
                        break  # Accept the new size
                    gate.current_size = original_size  # Revert if it didn't help

    def optimize_area_under_delay(self, delay_constraint, pog, Twall):
        rat_times = self.compute_required_arrival_times(delay_constraint, pog, Twall)
        self.go_to_local_min(rat_times)
        delays = self.compute_delays()
        arrival = self.compute_arrival_times(delays)
        slacks = self.compute_slacks(arrival, rat_times)

        print("\nFinal slacks before greedy adjustment:")
        for g, s in slacks.items():
            if s<0:
                self.gates[g].current_size +=1

        # Greedy adjustment based on slack values
        #self.greedy_slack_adjustment(slacks, rat_times)

        delays = self.compute_delays()
        arrival = self.compute_arrival_times(delays)
        slacks = self.compute_slacks(arrival, rat_times)

        print("\nFinal slacks after greedy adjustment:")
        for g, s in slacks.items():
            print(f"{g}: slack = {s:.2f}")

        return {g: self.gates[g].current_size for g in self.gates}, slacks

In [20]:
for gate_name in parser.gates:
    parser.gates[gate_name].current_size = size_dict[gate_name]
#parser = VerilogParser(verilog_code)
network = Network(parser.gates, parser,pog,Twall)
optimized_sizes = network.optimize_area_under_delay((1.29)*Twall,pog,Twall)

<ipython-input-19-47c622bc3640>:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return sum(
<ipython-input-19-47c622bc3640>:99: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  area = sum(self.gates[g].current_size for g in subnetwork)



Converged in 1 iterations.

Final slacks before greedy adjustment:

Final slacks after greedy adjustment:
NOT1_1: slack = 13.17
NOT1_2: slack = 51.05
NOT1_3: slack = 7.70
NAND2_4: slack = 6.77
NAND2_5: slack = 5.94
NAND2_6: slack = 5.94
NAND2_7: slack = 3.94
NAND2_8: slack = 3.81
NOT1_9: slack = 1.58
NOT1_10: slack = -0.14
NAND2_11: slack = 1.58
NAND2_12: slack = 0.17
NAND2_13: slack = 1.28
NOT1_14: slack = 43.74
NAND2_15: slack = 0.72
NAND2_16: slack = -0.22
NAND2_17: slack = 1.58
NAND2_18: slack = 0.82
NAND2_19: slack = 3.34
NAND2_20: slack = 12.24
NAND4_21: slack = 1.06
NAND4_22: slack = -0.59
NAND4_23: slack = -1.02
NAND4_24: slack = -0.00
NOT1_25: slack = 7.70
NAND4_26: slack = -0.44
NAND4_27: slack = 1.39
NAND4_28: slack = 1.65
NAND4_29: slack = 3.34
NAND4_30: slack = 12.24
NOT1_31: slack = 9.19
NOT1_32: slack = -0.91
NOT1_33: slack = 1.60
NOT1_34: slack = 2.44
NOT1_35: slack = 2.60
NOT1_36: slack = 43.98
NOR2_37: slack = -0.69
NAND2_38: slack = -0.56
NAND4_39: slack = -0.44
NAN